In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import os

data_dir = r"C:\Users\Safa Abubakar\Downloads\Alzheimer_MRI_4_classes_dataset-20250324T154737Z-001\Alzheimer_MRI_4_classes_dataset"
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)
model.save("efficientnet_alzheimer_model.h5")


Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
16705208/16705208 [==============================] - 120s 7us/step
Epoch 1/10
161/161 [==============================] - 415s 3s/step - loss: 1.0830 - accuracy: 0.4700 - val_loss: 1.0361 - val_accuracy: 0.5004
Epoch 2/10
161/161 [==============================] - 524s 3s/step - loss: 1.0604 - accuracy: 0.4964 - val_loss: 1.0340 - val_accuracy: 0.5004
Epoch 3/10
161/161 [==============================] - 578s 4s/step - loss: 1.0608 - accuracy: 0.4999 - val_loss: 1.0371 - val_accuracy: 0.5004
Epoch 4/10
161/161 [==============================] - 471s 3s/step - loss: 1.0620 - accuracy: 0.4999 - val_loss: 1.0360 - val_accuracy: 0.5004
Epoch 5/10
161/161 [==============================] - 478s 3s/step - loss: 1.0552 - accuracy: 0.4999 - val_loss: 1.0391 - val_accuracy: 0.5004
Epoch 6/10
161/161 [==============================] - 582s 4s/step - loss: 1.0513 - accuracy: 0.4999 - val_loss: 1.0410 - val_accuracy

C:\Users\Safa Abubakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
from sklearn.metrics import classification_report
import numpy as np
val_generator.reset()
y_true = val_generator.classes
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys()))

40/40 [==============================] - 69s 2s/step
Classification Report:
                  precision    recall  f1-score   support

    MildDemented       0.00      0.00      0.00       179
ModerateDemented       0.00      0.00      0.00        12
     NonDemented       0.50      1.00      0.67       640
VeryMildDemented       0.00      0.00      0.00       448

        accuracy                           0.50      1279
       macro avg       0.13      0.25      0.17      1279
    weighted avg       0.25      0.50      0.33      1279



C:\Users\Safa Abubakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Safa Abubakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Safa Abubakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklea